In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
# Image size and batch size
img_size = (128, 128)
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_gen = train_datagen.flow_from_directory(
    "Dataset_Split/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Validation set
val_gen = val_datagen.flow_from_directory(
    "Dataset_Split/val",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Test set
test_gen = test_datagen.flow_from_directory(
    "Dataset_Split/test",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  # Important for consistent evaluation
)

Found 3500 images belonging to 2 classes.
Found 750 images belonging to 2 classes.
Found 750 images belonging to 2 classes.


In [11]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

base_model = ResNet50(
    input_shape = (128,128,3),
    include_top = False,
    weights='imagenet'
)

base_model.trainable = False

base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False


model = Sequential(
    [
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

In [12]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:

early_stop = EarlyStopping(
    monitor='val_accuracy',     # You can also use 'val_loss' if that's more stable
    patience=3,                 # Wait for 3 epochs of no improvement
    restore_best_weights=True, # Restore the best model (not the last one)
    verbose=1                   # So it prints when it stops
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[early_stop]
)


Epoch 1/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 25s 228ms/step - accuracy: 0.9711 - loss: 0.0889 - val_accuracy: 0.9413 - val_loss: 0.1566
Epoch 2/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 200ms/step - accuracy: 0.9631 - loss: 0.1081 - val_accuracy: 0.7067 - val_loss: 2.8285
Epoch 3/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 201ms/step - accuracy: 0.9663 - loss: 0.0994 - val_accuracy: 0.7973 - val_loss: 0.6574
Epoch 4/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 23s 204ms/step - accuracy: 0.9674 - loss: 0.0949 - val_accuracy: 0.7693 - val_loss: 1.8733
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 1.


In [16]:

model.save('models/ResNet.keras')

In [17]:
test_loss, test_accuracy = model.evaluate(test_gen, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 439ms/step - accuracy: 0.9533 - loss: 0.1226
Test Accuracy: 95.33%
Test Loss: 0.1226
